In [1]:
from pytube import YouTube
from pytube import Playlist
from pydub import AudioSegment
from bs4 import BeautifulSoup
import requests
import os
from glob import glob
import openpyxl

### 列出 ID Title Duration URL

In [14]:
def DownloadInfoAndWav(label, youtube_lists_url, mp4_path, wav_path, workbook_path, worksheet_name, start=0, min_length=60):
    
    # 遍歷播放清單
    video_urls = []

    for yt_list_url in youtube_lists_url:

        # 播放列表URL
        playlist_url = yt_list_url

        # 創建Playlist對象
        playlist = Playlist(yt_list_url)

        # 解析播放列表
        playlist._video_regex = r"\"url\":\"(/watch\?v=[\w-]*)"

        # 播放列表URL
        video_urls += playlist.video_urls

    # 提取每首歌曲的編號、歌名、長度、URL
    song_titles = []
    song_ids = []
    song_durations = []
    song_urls = []
    song_i = start

    # 遍歷播放列表URL
    for i, url in enumerate(video_urls):

        # 儲存歌曲清單
        # 創建YouTube對象
        yt = YouTube(url)

        if (yt.length > min_length):
            song_i += 1
            song_titles.append(yt.title)
            song_ids.append(f"{label}_{song_i}")
            song_durations.append(yt.length)
            song_urls.append(url)

    # 印出每首歌曲的ID清單，並存進 xlsx
    workbook = openpyxl.load_workbook(workbook_path)
    sheet = workbook[worksheet_name]

    print("----List Of Song ID----")
    row = start+1
    for songId in song_ids:
        row += 1
        sheet[f'A{row}'] = songId
        print(songId)

    # 印出每首歌曲的標題清單
    print("----List Of Song Title----")
    row = start+1
    for title in song_titles:
        row += 1
        sheet[f'B{row}'] = title
        print(title)

    # 印出每首歌曲的長度清單
    print("----List Of Song Duration----")
    row = start+1
    for duration in song_durations:
        row += 1
        sheet[f'C{row}'] = duration
        print(duration)

    # 印出每首歌曲的URL清單
    print("----List Of Song URL----")
    row = start+1
    for url in song_urls:
        row += 1
        sheet[f'D{row}'] = url
        print(url)

    workbook.save(workbook_path)
    
    DownloadMp4ToWav(song_urls, song_ids, mp4_path, wav_path)
    
    return

### 下載 Youtube List 中每個影片

In [15]:
def DownloadMp4ToWav(song_urls, song_ids, mp4_path, wav_path):
    
    video_paths = []

    for i, url in enumerate(song_urls):
        try:
            # 創建YouTube對象
            youtube = YouTube(url)

            # 選擇最高質量的影片流
            video_stream = youtube.streams.get_highest_resolution()

            # 設定檔名
            file_name = f"{song_ids[i]}.mp4"

            # 下載影片
            video_file = video_stream.download(filename = file_name, 
                                               output_path = mp4_path)

            # 儲存影片路徑
            video_paths.append(f"{mp4_path}/{file_name}")

            print(f"成功下載: {youtube.title}")
        except Exception as e:
            print(f"下載失败: {str(e)}")

    # 印出每ㄧ MP4 的下載路徑清單
    print("List Of MP4 Path")
    for path in video_paths:
        print(path)
        
    for i, path in enumerate(video_paths):
    
        # 將 MP4 轉為 WAV 存起來
        mp4_audio = AudioSegment.from_file(path, "mp4")
        mp4_audio.export(f"{wav_path}/{song_ids[i]}.wav", format="wav")
        
        print(f"成功轉換: {song_ids[i]}.wav")
    
    # 刪除 MP4
    dir = mp4_path
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))
    
    return

In [28]:
# 設定參數和路徑
mp4_path = "/Users/phoebe/DJ-Box_model/dataset/mp4_files"


youtube_lists_url = ["https://youtube.com/playlist?list=PLNhzyV4NWnWNkxh5FK-AITO2R_3JGCbcr",
                    "https://youtube.com/playlist?list=PLNhzyV4NWnWMzmQlRL9iIQok-J0rR5fGS"]

classes = "occasion"
label = "awards"
wav_path = f"/Users/phoebe/DJ-Box_model/dataset/music_dataset/{classes}_set/{label}"

workbook_path = "/Users/phoebe/DJ-Box_model/dataset/music_dataset.xlsx"
worksheet_name = f"{classes}_{label}"

In [29]:
# 取得歌曲資訊
start = 0    # 開始編號
min_length = 60  # 最短長度

DownloadInfoAndWav(label, youtube_lists_url, mp4_path, wav_path, workbook_path, worksheet_name, start, min_length)

----List Of Song ID----
wedding_1
wedding_2
wedding_3
wedding_4
wedding_5
wedding_6
wedding_7
wedding_8
wedding_9
wedding_10
wedding_11
wedding_12
wedding_13
wedding_14
wedding_15
wedding_16
wedding_17
wedding_18
wedding_19
wedding_20
wedding_21
wedding_22
wedding_23
wedding_24
wedding_25
wedding_26
wedding_27
wedding_28
wedding_29
wedding_30
wedding_31
wedding_32
wedding_33
wedding_34
wedding_35
wedding_36
wedding_37
wedding_38
wedding_39
wedding_40
wedding_41
wedding_42
wedding_43
wedding_44
wedding_45
wedding_46
wedding_47
wedding_48
wedding_49
wedding_50
wedding_51
wedding_52
wedding_53
wedding_54
wedding_55
wedding_56
----List Of Song Title----
Perfect
Marry Me
Thinking Out Loud
Crispy脆樂團 [ 黑暗的盡頭 ] Official Music Video
I Love You So
Lucky
How Long Will I Love You (Official Video)
Kiss Me
不是因為天氣晴朗才愛你 (Instrumental)
Way Back Into Love
李千娜 Nana Lee – 心花開 (Official Music Video)
愛你
有你的快樂
A Rocket to the Moon
落日飛車-I Know You Know I Love You
旺福 Won Fu《我當你空氣 You Are My Air》 Official Music

成功轉換: wedding_1.wav
成功轉換: wedding_2.wav
成功轉換: wedding_3.wav
成功轉換: wedding_4.wav
成功轉換: wedding_5.wav
成功轉換: wedding_6.wav
成功轉換: wedding_7.wav
成功轉換: wedding_8.wav
成功轉換: wedding_9.wav
成功轉換: wedding_10.wav
成功轉換: wedding_11.wav
成功轉換: wedding_12.wav
成功轉換: wedding_13.wav
成功轉換: wedding_14.wav
成功轉換: wedding_15.wav
成功轉換: wedding_16.wav
成功轉換: wedding_17.wav
成功轉換: wedding_18.wav
成功轉換: wedding_19.wav
成功轉換: wedding_20.wav
成功轉換: wedding_21.wav
成功轉換: wedding_22.wav
成功轉換: wedding_23.wav
成功轉換: wedding_24.wav
成功轉換: wedding_25.wav
成功轉換: wedding_26.wav
成功轉換: wedding_27.wav
成功轉換: wedding_28.wav
成功轉換: wedding_29.wav
成功轉換: wedding_30.wav
成功轉換: wedding_31.wav
成功轉換: wedding_32.wav
成功轉換: wedding_33.wav
成功轉換: wedding_34.wav
成功轉換: wedding_35.wav
成功轉換: wedding_36.wav
成功轉換: wedding_37.wav
成功轉換: wedding_38.wav
成功轉換: wedding_39.wav
成功轉換: wedding_40.wav
成功轉換: wedding_41.wav
成功轉換: wedding_42.wav
成功轉換: wedding_43.wav
成功轉換: wedding_44.wav
成功轉換: wedding_45.wav
成功轉換: wedding_46.wav
成功轉換: wedding_47.wav
成功轉換: wedding_48.wav
成

In [6]:
# 刪除 MP4
dir = mp4_path
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))
dir = wav_path
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [ ]:
'''
youtube_lists_url = ["https://youtube.com/playlist?list=PLNhzyV4NWnWNFiDSn-pSV7_Q0JSxsE7oW",
                    "https://youtube.com/playlist?list=PLNhzyV4NWnWMb2xL7XFoEb2PKjnJBE4Mb"]

classes = "occasion"
label = "wedding"
'''

'''
youtube_lists_url = ["https://youtube.com/playlist?list=PLNhzyV4NWnWNOo9Hr8cNNnJbXf15n_S4t",
                    "https://youtube.com/playlist?list=PLNhzyV4NWnWNesLmJl4ejgChZ22_a79L1"]

classes = "occasion"
label = "sports"
'''

'''
youtube_lists_url = ["https://youtube.com/playlist?list=PLNhzyV4NWnWM3-Sb3nHeklp5-DPO-TAeA",
                    "https://youtube.com/playlist?list=PLNhzyV4NWnWOGniKUy8l9XUv88Cr9u08l"]

classes = "occasion"
label = "seminar"
'''

'''
youtube_lists_url = ["https://youtube.com/playlist?list=PLNhzyV4NWnWMgPFavWl5Eo1btzhoFyQi6",
                    "https://youtube.com/playlist?list=PLNhzyV4NWnWMWh4Y-6MaVc_8YTbRZWQaI"]

classes = "occasion"
label = "graduation"
'''

'''
youtube_lists_url = ["https://youtube.com/playlist?list=PLNhzyV4NWnWNYLfx4d5OqKaX-D4nBtCMC",
                    "https://youtube.com/playlist?list=PLNhzyV4NWnWOKgnqsVm1imvH8IQlMQlc2"]

classes = "occasion"
label = "gathering"
'''

'''
youtube_lists_url = ["https://youtube.com/playlist?list=PLNhzyV4NWnWOR7GeDM314wsmLD4wI31NR",
                    "https://youtube.com/playlist?list=PLNhzyV4NWnWMGOhMHQ5iN2kHsZxQcbt_D"]

classes = "occasion"
label = "company"
'''

'''
youtube_lists_url = ["https://youtube.com/playlist?list=PLNhzyV4NWnWMaEezVzVs9bBe4En0MYqq0",
                    "https://youtube.com/playlist?list=PLNhzyV4NWnWPg906ZXTcB8zekuhkTxswK"]

classes = "occasion"
label = "coffee"
'''

'''
youtube_lists_url = ["https://youtube.com/playlist?list=PLNhzyV4NWnWNkxh5FK-AITO2R_3JGCbcr",
                    "https://youtube.com/playlist?list=PLNhzyV4NWnWMzmQlRL9iIQok-J0rR5fGS"]

classes = "occasion"
label = "awards"
'''